In [203]:
# https://pyvis.readthedocs.io/en/latest/tutorial.html#getting-started
from pyvis.network import Network
import psycopg2
import pandas.io.sql as sqlio

In [204]:
conn = psycopg2.connect(
    dbname='dbname',
    user='dbuser',
    password='dbpwd',
    host='localhost',
    port='15432'
)
cursor = conn.cursor()

In [205]:
df = sqlio.read_sql_query("""
select h1.hashtag as hashtag_1, h2.hashtag as hashtag_2, hr.distance
from (select max(hr_tmp.hashtag_id_1) as hashtag_id_1, min(hr_tmp.hashtag_id_2) as hashtag_id_2, hr_tmp.distance
      from (select ph1.hashtag_id as hashtag_id_1, ph2.hashtag_id as hashtag_id_2, count(ph1.post_id) as distance
            from (select ph.*
                  from post_hashtag ph
                           join post p on p.id = ph.post_id
                  where p.main_tag = 'digitalclothing') ph1
                     join post_hashtag ph2 on ph1.post_id = ph2.post_id
            where ph1.hashtag_id != ph2.hashtag_id
            group by ph1.hashtag_id, ph2.hashtag_id
            order by distance DESC) hr_tmp
      group by distance, 1 / hr_tmp.hashtag_id_1 + 1 / hr_tmp.hashtag_id_2 + 1 / hr_tmp.distance
      order by hr_tmp.distance) hr
         join hashtag h1 on hr.hashtag_id_1 = h1.id
         join hashtag h2 on hr.hashtag_id_2 = h2.id
where hr.distance > 15
order by hr.distance DESC;
-- limit 14000;
""", conn)

In [206]:
display(df.head(10))

,hashtag_1,hashtag_2,distance
0,digitalclothing,digitalfashion,2357
1,digitalclothing,clo3d,1812
2,digitalclothing,virtualfashion,1376
3,digitalfashion,clo3d,1296
4,digitalclothing,3dfashion,1260
5,digitalclothing,fashion,1206
6,digitalfashion,virtualfashion,1166
7,digitalfashion,3dfashion,1043
8,marvelousdesigner,digitalclothing,1018
9,3dfashion,clo3d,951


In [207]:
got_net = Network(height='1200px', width='100%', bgcolor='#222222', font_color='white')
got_net.show_buttons()

In [208]:
# set the physics layout of the network
got_net.barnes_hut()

sources = df['hashtag_1']
targets = df['hashtag_2']
weights = df['distance']

edge_data = zip(sources, targets, weights)

In [209]:
for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

In [210]:
neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

In [211]:
got_net.show('insta_analytics_digitalclothing.html')